In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import re
import random
import os
import time
pattern = r"(?<=\d)([A-Za-z])"

In [ ]:
# Step 0: Configure Driver
options = Options()
options.binary_location = "/usr/bin/brave-browser"  # Adjust path as needed
driver = uc.Chrome(options=options)
driver.maximize_window()

# Step 1: Login to FetLife
login_url = "https://fetlife.com/login"
username = ""  # Replace with your username
password = ""  # Replace with your password
pages_which_failed = {
    "page": [],
    "error": []
}

try:
    driver.get(login_url)
    print("Opened FetLife login page.")

    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, "user_login")))
    print("Cloudflare challenge passed.")

    # Enter Login Credentials
    driver.find_element(By.ID, "user_login").send_keys(username)
    driver.find_element(By.ID, "user_password").send_keys(password)
    driver.find_element(By.XPATH, "//button[@type='submit']").click()
    print("Entered credentials and submitted login form.")

    # Verify Login Success
    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, "//span[contains(text(), 'Upload Pic')]")))
    print("Login successful and session active!")
except Exception as e:
    print(f"Login failed: {e}")
    driver.quit()
    exit()

In [ ]:
# Step 2: Navigate through All Pages of Kinksters
female_users = {}
current_page = 1
STATE = "texas"
CITY = "houston"
INTERESTED_GENDER = "F" #there are only 2 genders
# for some cities, its only CITY (without STATE)
try:
    # while current_page < 4:  # Loop through pages
    while True:
    # while current_page <= 512:  # Loop through pages

        kinksters_url = f"https://fetlife.com/p/united-states/{STATE}/{CITY}/kinksters?page={current_page}"
        driver.get(kinksters_url)
        print(f"Navigated to Kinksters page: {kinksters_url}")

        # Wait for the kinksters list to load
        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "[data-member-card]"))
            )
        except Exception as e:
            print(f"No more pages found or kinksters list not loaded: {e}")
            pages_which_failed["page"].append(current_page)
            pages_which_failed["error"].append(e)
            break

        # Scrape Female Users on the Current Page
        user_elements = driver.find_elements(By.CSS_SELECTOR, "[data-member-card]")
        if not user_elements:
            print("No more kinksters on this page.")
            break

        for user_element in user_elements:
            try:
                # Extract Username and Profile Link
                profile_link_element = user_element.find_element(By.CSS_SELECTOR, "a.link.text-red-500")
                profile_link = profile_link_element.get_attribute("href")
                username = profile_link_element.text

                # Extract User ID
                user_id = profile_link.split("/")[-1]

                # Extract Age/Gender
                age_gender_element = user_element.find_element(By.CSS_SELECTOR, "span.text-gray-300.font-bold")
                age_gender = age_gender_element.text

                # Check if user is female ("F")
                # if "F" in age_gender:
                genders = re.findall(pattern, age_gender)
                if INTERESTED_GENDER == genders[0]:
                    female_users[user_id] = {
                        "username": username,
                        "age_gender": age_gender,
                        "profile_link": profile_link,
                    }
            except Exception as e:
                print(f"Error processing user element: {e}")
                continue

        print(f"Completed scraping page {current_page}.")
        current_page += 1  # Move to the next page

    # Print Collected Female Users
    print("\nCollected Female Users:")
    for user_id, user_info in female_users.items():
        print(
            f"User ID: {user_id}, Username: {user_info['username']}, Age/Gender: {user_info['age_gender']}, Profile Link: {user_info['profile_link']}"
        )

except Exception as e:
    print(f"Error during pagination: {e}")
    driver.quit()
    exit()

In [ ]:
# Step 3: Collect User Data and Send Messages
# (Reuse the data collection and messaging logic from the previous steps.)

with open("female_users.json", "w") as user_file_id:
    json.dump(female_users, user_file_id)

# Step 3: Collect User Data
collected_users = {}
# read the female users from the file
with open("female_users.json", "r") as user_file:
    female_users = json.load(user_file)
if os.path.exists("sample_dataset.json"):
    with open("sample_dataset.json", "r") as user_file:
        collected_users = json.load(user_file)
if os.path.exists("pages_which_failed.json"):
    with open("pages_which_failed.json", "r") as pages_file:
        pages_which_failed = json.load(pages_file)
try:
    for i, (user_id, user_info) in enumerate(female_users.items()):
        if i % 10 == 0:
            with open("sample_dataset.json", "w") as user_file:
                json.dump(collected_users, user_file)
            with open("pages_which_failed.json", "w") as pages_file:
                json.dump(pages_which_failed, pages_file)
                # wait for 5 seconds
                # select a random number between 8 and 15
                # time.sleep()
                driver.implicitly_wait(random.randint(8, 15))
        try:
            # Skip users already processed
            # also check for their about != "No About section available."
            # if user_id in collected_users and collected_users[user_id]["about"] != "No About section available.":
            if user_id in collected_users:
                print(f"User already processed: {user_info['username']} (ID: {user_id})")
                continue
            print(f"Collecting data for user: {user_info['username']} (ID: {user_id})")

            # Navigate to the user's profile
            driver.get(user_info["profile_link"])
            print(f"Navigated to profile of {user_info['username']}.")

            # Extract "About" Section
            try:
                about_section_element = WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div.story__copy.relative.overflow-hidden"))
                )
                about_text = ""
                paragraph_elements = about_section_element.find_elements(By.TAG_NAME, "p")
                for paragraph in paragraph_elements:
                    about_text += paragraph.text + "\n"
                print(f"Extracted 'About' section for {user_info['username']}.")
                driver.implicitly_wait(random.randint(1, 3))
                
            except Exception as e:
                print(f"Could not find 'About' section for {user_info['username']}: {e}")
                about_text = "No About section available."

            # Store collected data
            collected_users[user_id] = {
                "username": user_info["username"],
                "profile_link": user_info["profile_link"],
                "about": about_text
            }

        except Exception as e:
            print(f"Error collecting data for user {user_info['username']} (ID: {user_id}): {e}")
            continue

    print("\nCollected User Data:")
    for user_id, user_data in collected_users.items():
        print(f"User ID: {user_id}, Username: {user_data['username']}, About: {user_data['about'][:100]}...")

except Exception as e:
    print(f"Error during data collection: {e}")
    driver.quit()
    exit()
with open("sample_dataset.json", "w") as user_file:
    json.dump(collected_users, user_file)
with open("pages_which_failed.json", "w") as pages_file:
    json.dump(pages_which_failed, pages_file)

In [ ]:
from tqdm import tqdm
import re
import torch
from transformers import pipeline
import json
torch.manual_seed(42)


model_id = "meta-llama/Llama-3.2-3B-Instruct"

pipe = pipeline(
		"text-generation", 
		model=model_id, 
		torch_dtype=torch.bfloat16, 
)
data_to_infer_path = "sample_dataset.json"
with open(data_to_infer_path, "r") as f:
		data = json.load(f)

for k, v in tqdm(data.items()):
	
		v["about"] = re.sub(r'[^\x00-\x7F]+', ' ', v["about"])
		about_her = v["about"]
		
		prompt = [
				{"role": "system",
				"content": (
						"You are a witty, charming, and playful individual who just discovered the profile of an intriguing and attractive woman on FetLife."
						"Your goal is to craft an engaging, personalized message that gets her attention and sparks a lively conversation. "
						"Be confident, use playful teasing and double entendres, and sprinkle in thoughtful compliments that show genuine interest in her profile. "
						"Maintain a light, flirtatious tone aiming to set the stage for a fun connection—whether for a date, a casual hookup, or simply good conversation. If the about section says somethine like: No About section available., just compliment her (How pretty she is) and ask for something casual like a dinner or talk more over a bottle of wine. Make it brief (as if written by someone in their late twenties), and no personal information about me and only output the response. "
				),
				},
				{"role": "user", "content": f"About her interests/kinks/fetishes, she has written: {about_her}"},
		]


		result = pipe(prompt, do_sample=False, top_p=1, max_new_tokens=400)
		temp_response = result[0]["generated_text"][-1]
		# get the "content" part of the response
		response = temp_response["content"]
		print(response)
		# update the response in the data
		v["response"] = response
		# save the updated data
		with open(data_to_infer_path, "w") as f:
				json.dump(data, f, indent=4)
		# break

In [ ]:
users_whom_i_have_messaged = []
WAIT_TIME = 20
if os.path.exists("users_whom_i_have_messaged.json"):
    with open("users_whom_i_have_messaged.json", "r") as user_file:
        users_whom_i_have_messaged = json.load(user_file)
# sample_dataset.json now contains the response
with open("sample_dataset.json", "r") as user_file:
    collected_users = json.load(user_file)
try:
    for user_id, user_data in collected_users.items():
        try:
            

            # Navigate to the conversation API endpoint
            conversation_url = f"https://fetlife.com/conversations/new?with={user_id}"
            driver.get(conversation_url)
            
            while True:
                try:
                    all_paragraphs = driver.find_elements(By.CSS_SELECTOR, "p.my-0.text-gray-200")
                    
                    # Check if any of the paragraphs contain the specific warning message
                    limit_reached = any(
                        "You've reached the maximum number of new conversations" in p.text for p in all_paragraphs
                    )
                    if limit_reached:
                        print("Reached the maximum number of new conversations. Waiting for 20 minutes.")
                        time.sleep(WAIT_TIME*60) # wait for 20 minutes
                    else:
                        break
                except Exception:
                    print("Limit not reached")
                    break
            print(f"Navigated to message form for {user_data['username']}.")
            driver.implicitly_wait(random.randint(2, 5))
            # Wait for the form fields to appear
            try:
                if user_data['username'] in users_whom_i_have_messaged:
                    print(f"Skipping user {user_data['username']} (ID: {user_id})")
                    continue
                print(f"Sending message to user: {user_data['username']} (ID: {user_id})")
                
                # Locate the subject field
                subject_field = WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.NAME, "chat[subject]"))
                )
                subject_field.clear()  # Clear any pre-filled text
                subject_field.send_keys("Hey Gorgeous")  # Enter the subject

                # Locate the body field
                body_field = WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.NAME, "chat[body]"))
                )
                body_field.clear()  # Clear any pre-filled text
                MESSAGE = user_data["response"]
                MESSAGE = MESSAGE[1:-1]
                body_field.send_keys(MESSAGE)  # Enter the message body
                driver.implicitly_wait(random.randint(3, 5))
                print(f"Entered subject and message for {user_data['username']}.")

                # Locate and click the "Start Conversation" button
                send_button = WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located(
                        (By.XPATH, "//button[@type='submit' and contains(text(), 'Start Conversation')]")
                    )
                )
                send_button.click()
                print(f"Message sent to {user_data['username']}.")
                users_whom_i_have_messaged.append(user_data['username'])
                driver.implicitly_wait(random.randint(9, 16))
                # save the users whom i have messaged
                with open("users_whom_i_have_messaged.json", "w") as user_send_msgs:
                    json.dump(users_whom_i_have_messaged, user_send_msgs)
                

            except Exception as e:
                print(f"Error interacting with message form for {user_data['username']}: {e}")
                continue

        except Exception as e:
            print(f"Error processing message for user {user_data['username']} (ID: {user_id}): {e}")
            continue

except Exception as e:
    print(f"Error during message sending: {e}")
    driver.quit()
    exit()
with open("users_whom_i_have_messaged.json", "w") as user_send_msgs:
    json.dump(users_whom_i_have_messaged, user_send_msgs)

# Finish
print("\nAll users processed successfully.")
driver.quit()